In [ ]:
import numpy as np

In [ ]:
import pertpy as pt

In [ ]:
import scanpy as sc

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import mudata as mu

In [ ]:
import sccoda.util.cell_composition_data as scc_dat
import sccoda.util.comp_ana as scc_ana
import sccoda.util.data_visualization as scc_viz

In [ ]:
import altair as alt

In [ ]:
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
tf.random.set_seed(0)

In [ ]:
sc.settings.set_figure_params(
    dpi=200,
    facecolor="white",
    frameon=False,
)

In [ ]:
adata = sc.read_h5ad("mapped_til_colon.h5ad")

In [ ]:
marker_dict_small  = {"CD8_NaiveLike":[
    "Ccr7",
"Il7r",
"Sell",
"Tcf7",
"Txk",
"S1pr1",
"Lef1",
"Satb1",
],"CD8_EarlyActiv":[
"Fos",
"Cd69",
"Zfp36",
"Fosb",
"Gzmm",
"Dusp2",
], "CD8_EffectorMemory":["Gzma",
"Gzmk",
"Cd8b1",
"Ctsw",
"Gzmb",
"Ccl5",
],"CD8_Tpex":[
"Xcl1",
"Crtam",
"Ifng",
"Ccl4",
"Dusp4",
"Zeb2",
"Nr4a2",
"Sla",
],"CD8_Tex":["Lag3",
"Prf1",
"Havcr2",
"Nkg7",
"Cd8b1",
"Ctsd",
"Klrd1",
"Id2",
"Cst7",
"Pdcd1",
"Tnfrsf9",
"Tigit"] }

In [ ]:
sc.pl.umap(adata, color = ["origin","condition","sample_id","cell_type",], legend_fontsize=7)

In [ ]:
sc.pl.matrixplot(adata, marker_dict_small, groupby='cell_type',gene_symbols="gene_name",dendrogram=True,  standard_scale ='var', cmap = "Reds" , save ="mapped_annotation_tumor_normal.svg")

In [ ]:
import os
os.getcwd()

In [ ]:
gtf = pd.read_csv("/data/scratch/kvalem/projects/2021/honda_microbial_metabolites_2021/20_scripts/40_single-cell-sorted-cd8/40_gex_surface_prot/18nov_analysis/tables/Mus_musculus.GRCm39.113.csv")
gtf = gtf.drop_duplicates(subset="gene_id")
gtf.rename(columns={"gene_id":"ensembl_id"}, inplace=True)
gtf.index =gtf.ensembl_id

In [ ]:
#adata.var = pd.merge(
#    adata.var, 
#    gtf, 
#    left_index=True,  # Use the index of adata.var
#    right_index=True, # Use the index of gtf_pc
#    how="left"        # Retain all rows from adata.var
#)
#adata.var = adata.var.rename(columns={
#    "ensembl_id_x": "ensembl_id", 
#    "gene_name_x": "gene_name"
#})
#
## Remove columns ensembl_id_y and gene_name_y
#adata.var = adata.var.drop(columns=["ensembl_id_y", "gene_name_y"], errors="ignore")
#subset = adata[:, adata.var["gene_biotype"]=="protein_coding"]

In [ ]:
adata

In [ ]:
adata_og = adata.copy()

In [ ]:
adata_og

In [ ]:
import numpy as np

# Check if 'Ifng' is in the variable names
if 'ENSMUSG00000055170' in adata_og.var_names:
    # Create a new column in adata.obs to store the expression status of Ifng
    adata_og.obs['Ifng_expression'] = adata_og[:, 'ENSMUSG00000055170'].X > 0
else:
    print("Gene 'Ifng' not found in adata.var_names")

In [ ]:
if 'ENSMUSG00000055170' in adata.var_names:
    # Create a new column in adata.obs to store the expression status of Ifng
    adata_og.obs['Ifng_expression'] = (adata_og[:, 'ENSMUSG00000055170'].X > 0).astype(int)
else:
    print("Gene 'Ifng' not found in adata.var_names")

In [ ]:
specific_order = [ "CD8_NaiveLike", "CD8_EarlyActiv", "CD8_EffectorMemory", "CD8_Tpex","CD8_Tex"]  # Replace with your desired order
adata_og.obs['cell_type'] = pd.Categorical(adata_og.obs['cell_type'], categories=specific_order, ordered=True)

In [ ]:
# Visualize
custom_colors = ["#3A923A", "#146aff", "#ff8113", "#fff314", "#D8393A"]  # Example colors

# Add these colors to the `adata_og.uns` dictionary for the `cell_type` category
adata_og.uns['cell_type_colors'] = custom_colors

sc.pl.umap(
    adata_og,
    color=["origin", "cell_type"],  # Add other variables you want to visualize
    legend_fontsize=8,
    frameon=False, save = "umap=colon_Til.svg"
)

In [ ]:
sc.pl.umap(
    adata_og,
    color=["condition", "sample_id","cell_type",],

    legend_fontsize=8,save = "umap_all.png"
)

In [ ]:
adata.obs.cell_type.value_counts()

In [ ]:
adata = adata[~adata.obs["cell_type"].isin(["nan"])]

In [ ]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, groupby="cell_type", method="wilcoxon")

In [ ]:
# Plot the dotplot
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="cell_type",
    standard_scale="var",
    n_genes=5,
    gene_symbols="gene_name",
   dendrogram=False,save="leiden_dotplot.png"
)


In [ ]:
import scanpy as sc

# Ensure that 'Ifng' exists in the data
if 'ENSMUSG00000055170' in adata.var_names:
    # Create a mask for cells with non-zero Ifng expression
    ifng_expressing_cells = adata[:, 'ENSMUSG00000055170'].X > 0
    
    # Subset the AnnData object
    adata = adata[ifng_expressing_cells].copy()
    
    print(f"Subset created with {adata.n_obs} cells expressing 'Ifng'.")
else:
    print("'Ifng' is not present in the dataset.")


In [ ]:
adata.obs

In [ ]:
# Visualize
sc.pl.umap(
    adata,
    color=["Ifng","cell_type", ],gene_symbols="gene_name",cmap="Reds",

    legend_fontsize=7,frameon=False
)

In [ ]:
categories_order= ["10mix1","10mix2","11mix1","11mix2","10mix_ICI1","10mix_ICI2","11mix_ICI1","11mix_ICI2"]

In [ ]:
sc.pl.dotplot(adata, "Ifng", "sample_id",gene_symbols="gene_name",log=True)

In [ ]:
# Visualize
sc.pl.umap(
    adata,
    color=["Ifng"],
vmax ="p99",gene_symbols="gene_name", cmap="Reds",
    legend_fontsize=8,frameon=False)#, save = "ifng_expressing_cells.png")

In [ ]:
adata_til = adata[adata.obs["origin"]=="til"]

In [ ]:
adata_til.obs.cell_type.value_counts()

In [ ]:
adata_til.obs.sample_id.value_counts()

In [ ]:
adata_til.obs.condition.value_counts()

In [ ]:
adata_colon = adata[adata.obs["origin"]=="colon"]

In [ ]:
adata = adata[adata.obs["sample_id"].isin(["10mix1","10mix2","11mix1","11mix2","10mix_ICI1","10mix_ICI2","11mix_ICI1","11mix_ICI2"])]

In [ ]:
#adata_colon.obs.cell_type.value_counts()

In [ ]:
adata.obs.sample_id.value_counts()

In [ ]:
adata.obs.condition.value_counts()

In [ ]:
adata.obs.origin.value_counts()

In [ ]:
adata.obs.cell_type.value_counts()

In [ ]:
adata.obs.sample_id.value_counts()

In [ ]:
categories_order= ["10mix1","10mix2","11mix1","11mix2","10mix_ICI1","10mix_ICI2","11mix_ICI1","11mix_ICI2"]

In [ ]:
sc.pl.matrixplot(adata, "Ifng", "sample_id",gene_symbols="gene_name",log=True,cmap="Reds")#, save ="10mix_11mix_expression.png")#,categories_order=categories_order)

In [ ]:
sc.pl.dotplot(adata, "Ifng", "sample_id",gene_symbols="gene_name",log=True,swap_axes=True,cmap="Reds")

In [ ]:
sc.pl.matrixplot(adata, "Ifng", "condition",gene_symbols="gene_name",log=True,swap_axes=True,cmap="Reds")#, save ="condition_xpression.png")#,categories_order=categories_order)

In [ ]:
specific_order = [ "CD8_NaiveLike", "CD8_EarlyActiv", "CD8_EffectorMemory", "CD8_Tpex","CD8_Tex"]  # Replace with your desired order
adata.obs['cell_type'] = pd.Categorical(adata.obs['cell_type'], categories=specific_order, ordered=True)
custom_colors = ["#3A923A", "#146aff", "#ff8113", "#fff314", "#D8393A"]  # Example colors

# Add these colors to the `adata_og.uns` dictionary for the `cell_type` category
adata.uns['cell_type_colors'] = custom_colors

sc.pl.umap(
    adata,
    color=["Ifng","sample_id"], groups = ["10mix1"],gene_symbols="gene_name",

    legend_fontsize=8,frameon=False)#, save = "ifng_new_anno.png")

In [ ]:
adata.obs.sample_id.value_counts()

In [ ]:
# Specify the desired order of the categories
desired_order = ["10mix1","10mix2","11mix1","11mix2","10mix_ICI1","10mix_ICI2","11mix_ICI1","11mix_ICI2"]

# Reorder the categories in adata.obs['leiden_res0_25']
adata.obs["sample_id"] = pd.Categorical(
    adata.obs["sample_id"],
    categories=desired_order,
    ordered=True
)


In [ ]:
adata

In [ ]:
adata_healty

In [ ]:
adata

In [ ]:
adata_healty = adata[adata.obs["sample_id"].isin(["10mix1","10mix2","11mix1","11mix2",])]
adata_tumor = adata[adata.obs["sample_id"].isin(["10mix_ICI1","10mix_ICI2","11mix_ICI1","11mix_ICI2"])]


In [ ]:

# Convert data to mudata object
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata_og,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="cell_type",
    sample_identifier="sample_id",
    covariate_obs=["condition","origin"],
)

# Select control and salmonella data as one modality
sccoda_data.mod["coda_til_11mix"] = sccoda_data["coda"][
    sccoda_data["coda"].obs["condition"].isin(["10mix", "11mix"])
].copy()
print(sccoda_data)



In [ ]:
sccoda_model.plot_boxplots(sccoda_data, modality_key="coda_til_11mix", feature_name="condition", add_dots=True)
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap  # Import ListedColormap

custom_colors = ["#E45756", "#4C78A8", "#F58518", "#54A24B", "#72b7b2","#cdff72"]

# Create a ListedColormap object
custom_cmap = ListedColormap(custom_colors)
# Create a ListedColormap object
custom_cmap = ListedColormap(custom_colors)
fig = sccoda_model.plot_stacked_barplot(
    sccoda_data,
    modality_key="coda_til_11mix",#,figsize=(2, 8),
    feature_name="condition",  palette=custom_cmap 

)

In [ ]:
from matplotlib.colors import ListedColormap  # Import ListedColormap

custom_colors = ["#1a9641", "#a6d96a", "#ffffbf", "#fdae61", "#d7191c", "#cdff72"]

sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="coda_til_11mix", feature_name="sample_id",    figsize=(3, 8),  palette=custom_cmap )
plt.show()
#plt.savefig("./figures/barplot_ifng_all_sample_id.png", dpi=300, bbox_inches="tight")       

In [ ]:
from matplotlib.colors import ListedColormap  # Import ListedColormap

custom_colors = ["#1a9641", "#a6d96a", "#ffffbf", "#fdae61", "#d7191c","#cdff72"]

sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="coda_til_11mix", feature_name="sample_id",    figsize=(3, 8),  palette=custom_cmap )
plt.show()
#plt.savefig("./figures/barplot_ifng_all_sample_id.png", dpi=300, bbox_inches="tight")       

In [ ]:
from matplotlib.colors import ListedColormap  # Import ListedColormap

custom_colors = ["#1a9641", "#a6d96a", "#ffffbf", "#fdae61", "#d7191c","#cdff72"]

sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="coda_til_11mix", feature_name="origin",    figsize=(2, 8), palette=custom_cmap )
plt.show()
#plt.savefig("./figures/barplot_ifng_all_origin.png", dpi=300, bbox_inches="tight")       

In [ ]:
from matplotlib.colors import ListedColormap  # Import ListedColormap

custom_colors = ["yellow", "#4C78A8", "#F58518", "#54A24B", "#72b7b2","#cdff72"]
sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="coda_til_11mix", feature_name="condition",    figsize=(2, 8), palette=custom_cmap )
plt.show()
#plt.savefig("./figures/barplot_ifng_all_origin.png", dpi=300, bbox_inches="tight")       

In [ ]:
sccoda_model.plot_rel_abundance_dispersion_plot(sccoda_data, modality_key="coda_til_11mix", abundant_threshold=0.9)

In [ ]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda_til_11mix",
    formula="condition + origin",
    reference_cell_type="automatic",
    automatic_reference_absence_threshold=1
)
sccoda_data["coda_til_11mix"]

In [ ]:
# Run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda_til_11mix")
sccoda_data["coda_til_11mix"]

In [ ]:
adata.obs.sample_id.value_counts()

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="coda_til_11mix", est_fdr=0.3)

In [ ]:
sccoda_model.summary(sccoda_data, modality_key="coda_til_11mix")

In [ ]:
sccoda_model.plot_effects_barplot(sccoda_data, modality_key="coda_til_11mix", parameter="log2-fold change",figsize=(10,5))

In [ ]:
sccoda_model.get_effect_df(sccoda_data, modality_key="coda_til_11mix")

In [ ]:
alt.Chart(sccoda_model.get_effect_df(sccoda_data, modality_key="coda_til_11mix").loc["conditionT.11mix"]
    #.loc[credible_effects_condition]
    .reset_index(),
    title="11mix vs 10mix",
).mark_bar().encode(
    x="log2-fold change",
    y=alt.X("Cell Type", sort="y"),
    color=alt.Color("Cell Type"),
)

In [ ]:
from matplotlib.colors import ListedColormap  # Import ListedColormap

custom_colors = ["#E45756","#4C78A8","#F58518","#54A24B", "#72b7b2",]

# Create a ListedColormap object
custom_cmap = ListedColormap(custom_colors)
# Create a ListedColormap object
custom_cmap = ListedColormap(custom_colors)
fig = sccoda_model.plot_stacked_barplot(
    sccoda_data,
    modality_key="coda_til_11mix",figsize=(2, 8),
    feature_name="condition",  palette=custom_cmap 

)

In [ ]:
alt.Chart(sccoda_model.get_effect_df(sccoda_data, modality_key="coda_til_11mix").loc["originT.til"]
    #.loc[credible_effects_condition]
    .reset_index(),
    title="Tumor vs Normal",
).mark_bar().encode(
    x="log2-fold change",
    y=alt.X("Cell Type", sort="y"),
   # color=alt.Color("Cell Type"),
)

In [ ]:
## comparison ifng vs no ifng in 11mix 10mix 

In [ ]:
adata_subset = adata_og[adata_og.obs["sample_id"].isin(["10mix1","10mix2","11mix1","11mix2","10mix_ICI1","10mix_ICI2","11mix_ICI1","11mix_ICI2"])]

In [ ]:
if 'ENSMUSG00000055170' in adata_subset.var_names:
    # Safely extract the expression values for 'Ifng' and ensure alignment with `adata.obs`
    ifng_expression = (
        adata_subset[:, 'ENSMUSG00000055170'].X.toarray().flatten()
        if hasattr(adata_subset[:, 'ENSMUSG00000055170'].X, "toarray")
        else adadata_subsetata[:, 'ENSMUSG00000055170'].X.flatten()
    )
       # Add Ifng expression status to adata.obs
    adata_subset.obs['Ifng_expression'] = (ifng_expression > 0).astype(int)

     # Create a DataFrame for plotting
    plot_df = adata.obs.copy()
    plot_df['Ifng_expression'] = plot_df['Ifng_expression'].map({1: 'Expressing Ifng', 0: 'Not Expressing Ifng'})

    
    # Ensure `cell_type` is in adata.obs (or adjust based on your column name)
    if 'cell_type' not in adata_subset.obs:
        raise ValueError("`cell_type` column not found in adata.obs. Please ensure it exists.")

    # Count the proportions of each cell type in the two groups
    comp_df = (
        plot_df.groupby(['Ifng_expression', 'cell_type'])
        .size()
        .reset_index(name='count')
    )
    #comp_df['proportion'] = comp_df.groupby('Ifng_expression')['count'].apply(lambda x: x / x.sum())




In [ ]:
comp_df

In [ ]:
#comp_df['proportion'] = comp_df.groupby('Ifng_expression')['count']

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Check if 'Ifng' is in the variable names
if 'ENSMUSG00000055170' in adata_subset.var_names:
    # Safely extract the expression values for 'Ifng' and ensure alignment with `adata.obs`
    ifng_expression = (
        adata_subset[:, 'ENSMUSG00000055170'].X.toarray().flatten()
        if hasattr(adata_subset[:, 'ENSMUSG00000055170'].X, "toarray")
        else adadata_subsetata[:, 'ENSMUSG00000055170'].X.flatten()
    )
    
    # Add Ifng expression status to adata.obs
    adata_subset.obs['Ifng_expression'] = (ifng_expression > 0).astype(int)

    # Create a DataFrame for plotting
    plot_df = adata_subset.obs.copy()
    plot_df['Ifng_expression'] = plot_df['Ifng_expression'].map({1: 'Expressing Ifng', 0: 'Not Expressing Ifng'})

    # Ensure `cell_type` is in adata.obs (or adjust based on your column name)
    if 'cell_type' not in adata_subset.obs:
        raise ValueError("`cell_type` column not found in adata.obs. Please ensure it exists.")

    # Count the proportions of each cell type in the two groups
    comp_df = (
        plot_df.groupby(['Ifng_expression', 'cell_type_2'])
        .size()
        .reset_index(name='count')
    )
    #comp_df['proportion'] = comp_df.groupby('Ifng_expression')['count'].apply(lambda x: x / x.sum())

    # Create the compositional bar plot
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=comp_df,
        x='Ifng_expression',
        y='count',
        hue='cell_type_2',
        palette='tab10'
    )
    plt.title('Compositional Bar Plot of Cell Types by Ifng Expression', fontsize=14)
    plt.xlabel('Ifng Expression', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("Gene 'Ifng' not found in adata.var_names")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Check if 'Ifng' is in the variable names
if 'ENSMUSG00000055170' in adata_subset.var_names:
    # Safely extract the expression values for 'Ifng' and ensure alignment with `adata.obs`
    ifng_expression = (
        adata_subset[:, 'ENSMUSG00000055170'].X.toarray().flatten()
        if hasattr(adata_subset[:, 'ENSMUSG00000055170'].X, "toarray")
        else adadata_subsetata[:, 'ENSMUSG00000055170'].X.flatten()
    )
    
    # Add Ifng expression status to adata.obs
    adata_subset.obs['Ifng_expression'] = (ifng_expression > 0).astype(int)

    # Create a DataFrame for plotting
    plot_df = adata.obs.copy()
    plot_df['Ifng_expression'] = plot_df['Ifng_expression'].map({1: 'Expressing Ifng', 0: 'Not Expressing Ifng'})

    # Ensure `cell_type` is in adata.obs (or adjust based on your column name)
    if 'cell_type' not in adata_subset.obs:
        raise ValueError("`cell_type` column not found in adata.obs. Please ensure it exists.")

    # Count the proportions of each cell type in the two groups
    comp_df = (
        plot_df.groupby(['Ifng_expression', 'condition'])
        .size()
        .reset_index(name='count')
    )
    #comp_df['proportion'] = comp_df.groupby('Ifng_expression')['count'].apply(lambda x: x / x.sum())

    # Create the compositional bar plot
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=comp_df,
        x='Ifng_expression',
        y='count',
        hue='condition',
        palette='tab10'
    )
    plt.title('Compositional Bar Plot of Cell Types by Ifng Expression', fontsize=14)
    plt.xlabel('Ifng Expression', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Cell Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("Gene 'Ifng' not found in adata.var_names")


In [ ]:
adata_og.obs

In [ ]:
sc.pl.umap(adata_og, color =["cell_type_2"], gene_symbols="gene_name")

In [ ]:
# Check if 'Ifng' is in the variable names
if 'ENSMUSG00000055170'in adata.var_names:
    # Safely extract the expression values for 'Ifng'
    ifng_expression = (
        adata[:, 'ENSMUSG00000055170'].X.toarray().flatten()
        if hasattr(adata[:, 'ENSMUSG00000055170'].X, "toarray")
        else adata[:, 'ENSMUSG00000055170'].X.flatten()
    )
    
    # Add Ifng expression status to adata.obs
    adata.obs['Ifng_expression'] = (ifng_expression > 6).astype(int)
    
    # Create a new column 'cell_type_2' based on 'Ifng_expression' and 'cell_type'
    adata.obs['cell_type_2'] = adata.obs.apply(
        lambda row: 'Ifng+' if row['Ifng_expression'] == 1 else row['cell_type'], axis=1
    )
else:
    print("Gene 'Ifng' not found in adata.var_names")


In [ ]:
# Check if 'Ifng' is in the variable names
if  'ENSMUSG00000055170' in adata_og.var_names:
    # Safely extract the expression values for 'Ifng'
    ifng_expression = (
        adata_og[:,  'ENSMUSG00000055170'].X.toarray().flatten()
        if hasattr(adata_og[:,  'ENSMUSG00000055170'].X, "toarray")
        else adata_og[:,  'ENSMUSG00000055170'].X.flatten()
    )
    
    # Add Ifng expression status to adata.obs
    adata_og.obs['Ifng_expression'] = (ifng_expression > 4).astype(int)
    # Create a new column 'cell_type_2' based on 'Ifng_expression' and 'cell_type'
    adata_og.obs['cell_type_2'] = adata_og.obs.apply(
        lambda row: 'Ifng+' if row['Ifng_expression'] == 1 else row['cell_type'], axis=1)
else:
    print("Gene 'Ifng' not found in adata.var_names")


In [ ]:
adata_og.obs.cell_type_2.value_counts()

In [ ]:
# Visualize
sc.pl.umap(
    adata_og,
    color=["cell_type_2"], groups = ["Ifng+"],
vmax ="p99",gene_symbols="gene_name", cmap="Reds",
    legend_fontsize=8,frameon=False)#, save = "ifng_expressing_cells.png")

In [ ]:
adata

In [ ]:
sccoda_data